Goal of this notebook is to finalize features and examples into arrays for train, train-val, val, and test sets with shuffled datasets, normalization, splitting datasets, and labels to match

In [1]:
# Import Libraries and Set Dependencies
import importlib, os, sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.core.display import display
from icecream import ic

sys.path.append('../')
sys.path.append('/Volumes/Lab/Users/scooler/classification/')
sys.path.append("/Volumes/Lab/Users/mads/artificial-retina-software-pipeline/artificial-retina-software-pipeline/utilities/")
sys.path.append("/Volumes/Lab/Users/mads/cell_class/moosa_share/")

import cell_display_lib as cdl
import features as feat
import features_visual as feat_v
import features_electrical as feat_e
import deduplication
import features_DLelec as feat_dl

import features
import file_handling
from scipy.signal import spectrogram 
import scipy.signal as signal
import plotly.express as px
from sklearn.decomposition import PCA
import visionloader as vl
from conduction_velocity_code import get_axonal_conduction_velocity, upsample_ei, filter_ei_for_electrode_types
from scipy import stats
import eilib as el
import math
from skimage import measure
from cell_display_lib import show
import cv2
import re

import glob
import resampy
import random
pd.set_option('display.max_rows', 4000)



/tmp/ipykernel_17914/3889554625.py:6: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display


## Load all possible datasets for train & train-val

In [2]:
# Looks for all the large dataset files, loads the piece information and concatenates to form dataset, cell, and unit tables
import glob
scratch_file_root = '/Volumes/Scratch/Users/mads/celltable_datasets/featExtractDL/train/'
ind = 0
for file in glob.glob(scratch_file_root+ 'ctds_*'):
    print(file)
    
    dt = pd.read_pickle(file)
    if ind == 0:
        dataset_tablet = dt
    else:
        dataset_tablet = pd.concat([dataset_tablet, dt],ignore_index = True)
        # dataset_table.append(dt)
        
    pieces = np.array(dt.piece_id.unique())
    for pp, piece_id in enumerate(pieces):
        file_name = (scratch_file_root + 'ctd_' + piece_id)
        # ds = pd.read_pickle(file_name + '_d.pkl')
        cs = pd.read_pickle(file_name + '_c.pkl')
        us = pd.read_pickle(file_name + '_u.pkl')
        if pp == 0 and ind == 0:
            ctt = cs
            unit_tablet = us
        else:
            ctt = pd.concat([ctt, cs],ignore_index = True)
            unit_tablet = pd.concat([unit_tablet, us],ignore_index = True)
    ind += 1



/Volumes/Scratch/Users/mads/celltable_datasets/featExtractDL/train/ctds_train240to260.pkl
/Volumes/Scratch/Users/mads/celltable_datasets/featExtractDL/train/ctds_train180to200.pkl
/Volumes/Scratch/Users/mads/celltable_datasets/featExtractDL/train/ctds_train80to100.pkl
/Volumes/Scratch/Users/mads/celltable_datasets/featExtractDL/train/ctds_train160to180.pkl
/Volumes/Scratch/Users/mads/celltable_datasets/featExtractDL/train/ctds_train120to140.pkl
/Volumes/Scratch/Users/mads/celltable_datasets/featExtractDL/train/ctds_train200to220.pkl
/Volumes/Scratch/Users/mads/celltable_datasets/featExtractDL/train/ctds_train360to364.pkl
/Volumes/Scratch/Users/mads/celltable_datasets/featExtractDL/train/ctds_train320to340.pkl
/Volumes/Scratch/Users/mads/celltable_datasets/featExtractDL/train/ctds_train20to40.pkl
/Volumes/Scratch/Users/mads/celltable_datasets/featExtractDL/train/ctds_train60to80.pkl
/Volumes/Scratch/Users/mads/celltable_datasets/featExtractDL/train/ctds_train280to300.pkl
/Volumes/Scratc

In [3]:
# Check current number of datasets and cells
print(len(dataset_tablet),len(unit_tablet))

364 162051


## Load all possible datasets for test & test-val

In [4]:
# Looks for all the large dataset files, loads the piece information and concatenates to form dataset, cell, and unit tables
import glob
scratch_file_root = '/Volumes/Scratch/Users/mads/celltable_datasets/featExtractDL/test/'
ind = 0
for file in glob.glob(scratch_file_root+ 'ctds_*'):
    print(file)
    
    dt = pd.read_pickle(file)
    if ind == 0:
        dataset_table = dt
    else:
        dataset_table = pd.concat([dataset_table, dt],ignore_index = True)
        # dataset_table.append(dt)
        
    pieces = np.array(dt.piece_id.unique())
    for pp, piece_id in enumerate(pieces):
        file_name = (scratch_file_root + 'ctd_' + piece_id)
        # ds = pd.read_pickle(file_name + '_d.pkl')
        cs = pd.read_pickle(file_name + '_c.pkl')
        us = pd.read_pickle(file_name + '_u.pkl')
        if pp == 0 and ind == 0:
            ct = cs
            unit_table = us
        else:
            ct = pd.concat([ct, cs],ignore_index = True)
            unit_table = pd.concat([unit_table, us],ignore_index = True)
    ind += 1


/Volumes/Scratch/Users/mads/celltable_datasets/featExtractDL/test/ctds_test30to45.pkl
/Volumes/Scratch/Users/mads/celltable_datasets/featExtractDL/test/ctds_test45to52.pkl
/Volumes/Scratch/Users/mads/celltable_datasets/featExtractDL/test/ctds_test0to15.pkl
/Volumes/Scratch/Users/mads/celltable_datasets/featExtractDL/test/ctds_test15to30.pkl


In [5]:
# Check current number of datasets and cells
print(len(dataset_table),len(unit_table))

52 31217


## Only Keep the 5 Major Cell Types

In [6]:
# Only keep the 5 major cell types for our unit table of interest (easy to change)
unit_tablet = unit_tablet.loc[unit_tablet['label_manual_text'].isin(['SBC','ON parasol','OFF parasol','ON midget','OFF midget'])]
unit_table = unit_table.loc[unit_table['label_manual_text'].isin(['SBC','ON parasol','OFF parasol','ON midget','OFF midget'])]

In [7]:
# Number of cells left
print(len(unit_tablet),len(unit_table))

118949 20897


## Lists for Which Datasets in train, training-validation, validation, and test

In [8]:
# Determine which pieces will be used in training set, training-validation set, validation set, and test set
pos_testvalsets = dataset_table['piece_id'].tolist()
random.shuffle(pos_testvalsets)
train_testvalsets = pos_testvalsets[:10]
test_testvalsets = pos_testvalsets[10:31]
val_testvalsets = pos_testvalsets[31:]

pos_trainvalsets = dataset_tablet['piece_id'].tolist()
pos_trainvalsets.append(train_testvalsets)
random.shuffle(pos_trainvalsets)
trainval_trainvalsets = pos_trainvalsets[:11]
train_trainvalsets = pos_trainvalsets[11:]

## Make Full Dataset

In [9]:
# Combine all units into one full dataset
all_units = pd.concat([unit_table,unit_tablet],ignore_index=True)

## Make Sure Dimensions are the Same for All Features of the Dataset

In [10]:
# For any undersampled time series, interpolate to fill them in
# Features to Check Sizing: acf, spike waveform max amplitude, spec spike waveform, spec freq waveform, spec acf, 
# spec freq acf
# Loop threw values to check
check = ['acf', 'spike_waveform_maxamplitude', 'spec_spike_waveform', 'spec_freq', 'spec_acf', 'spec_freq_acf']

for ind in check:
    # Find max value
    isolate = pd.Series(all_units[ind])
    maxval = max(isolate.apply(lambda x: len(x.a.flatten())))
    # argmaxval = np.argmax(isolate.apply(lambda x: len(x.a)))
    print(ind)
    print('Max Value:' + str(maxval))
    minval = min(isolate.apply(lambda x: len(x.a.flatten())))
    # argminval = np.argmin(isolate.apply(lambda x: len(x.a)))
    print('Min Value:' + str(minval))
    
    for index, row in all_units.iterrows():
        if len(all_units[ind][index].a.flatten()) < maxval:
            FS = len(all_units[ind].iloc[index].a.flatten()) # samples per ms
            all_units[ind].iloc[index].a = resampy.resample(all_units[ind].iloc[index].a.flatten(),FS,maxval)
            # unit_tablet[ind][index].a = resampy.resample(unit_tablet[ind][index].a,FS,maxval)
        elif len(all_units[ind][index].a.flatten()) > maxval:
            print('max calc fail')
    maxval = max(isolate.apply(lambda x: len(x.a.flatten())))
    print('New Max Value:' + str(maxval))
    minval = min(isolate.apply(lambda x: len(x.a.flatten())))
    print('New Min Value:' + str(minval))



acf
Max Value:100
Min Value:100
New Max Value:100
New Min Value:100
spike_waveform_maxamplitude
Max Value:181
Min Value:51
New Max Value:181
New Min Value:181
spec_spike_waveform
Max Value:91
Min Value:26
New Max Value:91
New Min Value:91
spec_freq
Max Value:91
Min Value:26
New Max Value:91
New Min Value:91
spec_acf
Max Value:51
Min Value:51
New Max Value:51
New Min Value:51
spec_freq_acf
Max Value:51
Min Value:51
New Max Value:51
New Min Value:51


## Split between Training/Training-Validation Set and Validation/Testing Set

In [11]:
trainingvalsets = np.append(trainval_trainvalsets, train_trainvalsets)
valtestsets = np.append(val_testvalsets, test_testvalsets)
trainingval = all_units.loc[all_units['piece_id'].isin(trainingvalsets)]
valtesting = all_units.loc[all_units['piece_id'].isin(valtestsets)]

/Volumes/Lab/Development/miniconda-peggyo/envs/mads3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:1822: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asanyarray(a).ravel(order=order)


## Normalize Training/Training Validation Sets

In [12]:
# Features to Normalize: pieceid, runid, spike duration, spike_rate_mean,retinal eccentricity, total energy,
# early area, early peri,  early circularity, soma area, soma peri, 
# soma circularity, spike min h, spike max l, spike minmax ratio, spike width half min amp, 
# spike width half max amp, spike area amp upper, spike area amp lower, spike area tr amp upper, 
# spike area tr amp lower, spike slope amp upper, spike slopw amp lower, spike area2, axon vel
norm = ['int_piece_id', 'int_run_id', 'spike_duration', 'spike_rate_mean', 'retinal_eccentricity',
        'total_energy_ptnorm', 'early_area', 'early_peri', 'early_circularity',
        'soma_area', 'soma_peri', 'soma_circularity', 'spike_min_h', 'spike_max_l',
        'spike_minmax_ratio', 'spike_width_half_min_amp', 'spike_width_half_max_amp', 
        'spike_area_amp_upper', 'spike_area_amp_lower', 'spike_area_tr_amp_upper', 
        'spike_area_tr_amp_lower', 'spike_area2', 'axon_vel']

# Future fix, piece id for experiments with 10 or more pieces
mean_orig = []
std_orig = []
for ind in norm:
    trainingval[ind] = trainingval[ind].replace(np.nan,0)
    if isinstance(trainingval[ind].iloc[0],str):
        trainingval[ind] = trainingval[ind].apply(lambda x: pd.to_numeric(x, errors='coerce'))
        trainingval[ind] = trainingval[ind].replace(np.nan,0)
    isolate = pd.to_numeric(pd.Series(trainingval[ind]))
    mean = isolate.mean()
    mean_orig.append(mean)
    std = isolate.std()
    std_orig.append(std)
    print(ind, mean, std)
    trainingval[ind] = trainingval[ind].apply(lambda x: (x-mean)/std)

/tmp/ipykernel_17914/959375583.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trainingval[ind] = trainingval[ind].replace(np.nan,0)
/tmp/ipykernel_17914/959375583.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trainingval[ind] = trainingval[ind].apply(lambda x: (x-mean)/std)


int_piece_id 205912696.03541854 93464139.67412134
int_run_id 5.214562543611128 10.334919221980384
spike_duration 1961.46574445268 1416.0678369435211
spike_rate_mean 8.643154161811479 6.843364116284916


/tmp/ipykernel_17914/959375583.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trainingval[ind] = trainingval[ind].apply(lambda x: pd.to_numeric(x, errors='coerce'))
/tmp/ipykernel_17914/959375583.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trainingval[ind] = trainingval[ind].replace(np.nan,0)


retinal_eccentricity 9.95070576465544 13.811397667466162
total_energy_ptnorm -0.47322464079653936 1.0863931287531015
early_area 84.64295202145458 74.59593236969235
early_peri 52.29307268544461 34.68428587409382
early_circularity 0.34175932898028405 0.1981994493874022
soma_area 2.580320137201658 1.2067695909704383
soma_peri 7.089723772095203 1.9039958909934622
soma_circularity 0.6038399547437479 0.16647480823182012
spike_min_h 192.96990268421592 142.97555311422082
spike_max_l 111.1448158887165 100.51678638132209
spike_minmax_ratio 0.5654952647109163 4.888400184758875
spike_width_half_min_amp 46.06658315748766 14.469028936919868
spike_width_half_max_amp 101.65943387502207 39.793716380281836
spike_area_amp_upper 3733.055642655824 2950.3985405894455
spike_area_amp_lower 3228.250355711798 2556.567080090794
spike_area_tr_amp_upper 3426.4879419093495 2675.0928609040147
spike_area_tr_amp_lower 3115.481877481664 2426.956580595799
spike_area2 8334.298099371097 7615.90861221812
axon_vel 14.099974

## Normalize Validation/Testing Sets

In [13]:
norm = ['int_piece_id', 'int_run_id', 'spike_duration', 'spike_rate_mean', 'retinal_eccentricity',
        'total_energy_ptnorm', 'early_area', 'early_peri', 'early_circularity',
        'soma_area', 'soma_peri', 'soma_circularity', 'spike_min_h', 'spike_max_l',
        'spike_minmax_ratio', 'spike_width_half_min_amp', 'spike_width_half_max_amp', 
        'spike_area_amp_upper', 'spike_area_amp_lower', 'spike_area_tr_amp_upper', 
        'spike_area_tr_amp_lower', 'spike_area2', 'axon_vel']

# Future fix, piece id for experiments with 10 or more pieces
index = 0
for ind in norm:
    valtesting[ind] = valtesting[ind].replace(np.nan,0)
    if isinstance(valtesting[ind].iloc[0],str):
        valtesting[ind] = valtesting[ind].apply(lambda x: pd.to_numeric(x, errors='coerce'))
        valtesting[ind] = valtesting[ind].replace(np.nan,0)
    
    valtesting[ind] = valtesting[ind].apply(lambda x: (x-mean_orig[index])/std_orig[index])
    index += 1



/tmp/ipykernel_17914/2615685640.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valtesting[ind] = valtesting[ind].replace(np.nan,0)
/tmp/ipykernel_17914/2615685640.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valtesting[ind] = valtesting[ind].apply(lambda x: (x-mean_orig[index])/std_orig[index])
/tmp/ipykernel_17914/2615685640.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

## Make Example & Label Arrays for Each Set

In [46]:
# Labels Here:
testpos_labels = valtesting['label_manual_text'][valtesting['piece_id'].isin(test_testvalsets)].tolist()
valpos_labels = valtesting['label_manual_text'][valtesting['piece_id'].isin(val_testvalsets)].tolist()
trainval_labels = trainingval['label_manual_text'][trainingval['piece_id'].isin(trainval_trainvalsets)].tolist()
train_labels = trainingval['label_manual_text'][trainingval['piece_id'].isin(train_trainvalsets)].tolist()

# Single Number Features for Examples Here:
singfeat = ['spike_duration','spike_rate_mean','int_piece_id', 'int_run_id','retinal_eccentricity',
        'total_energy_ptnorm', 'early_area', 'early_peri', 'early_circularity',
        'soma_area', 'soma_peri', 'soma_circularity', 'spike_min_h', 'spike_max_l',
        'spike_minmax_ratio', 'spike_width_half_min_amp', 'spike_width_half_max_amp', 
        'spike_area_amp_upper', 'spike_area_amp_lower', 'spike_area_tr_amp_upper', 
        'spike_area_tr_amp_lower', 'spike_area2', 'axon_vel']

arrfeat = ['acf', 'spike_waveform_maxamplitude', 'spec_spike_waveform', 'spec_freq', 'spec_acf', 
           'spec_freq_acf', 'early_centroid','soma_centroid']

test_feat = valtesting[singfeat][valtesting['piece_id'].isin(test_testvalsets)].to_numpy()
val_feat = valtesting[singfeat][valtesting['piece_id'].isin(val_testvalsets)].to_numpy()
trainval_feat = trainingval[singfeat][trainingval['piece_id'].isin(trainval_trainvalsets)].to_numpy()
train_feat = trainingval[singfeat][trainingval['piece_id'].isin(train_trainvalsets)].to_numpy()

# Time Series Features Here:

for ind in arrfeat:
    print(ind)
    if ind == 'early_centroid' or ind == 'soma_centroid':
        isolate1 = pd.Series(valtesting[ind]).apply(lambda x: np.asarray(x))
        isolate2 = pd.Series(trainingval[ind]).apply(lambda x: np.asarray(x))
        
        seriesA = isolate1[valtesting['piece_id'].isin(test_testvalsets)].apply(lambda x: np.asarray(x.a))
        array = seriesA.to_numpy()
        final = np.vstack(array)
        test_feat = np.concatenate((test_feat, final),axis=1)
        print('test done')
        
        seriesA = isolate1[valtesting['piece_id'].isin(val_testvalsets)].apply(lambda x: np.asarray(x.a))
        array = seriesA.to_numpy()
        final = np.vstack(array)
        val_feat = np.concatenate((val_feat, final),axis=1)
        print('val done')

        seriesA = isolate2[trainingval['piece_id'].isin(trainval_trainvalsets)].apply(lambda x: np.asarray(x.a))
        array = seriesA.to_numpy()
        final = np.vstack(array)
        trainval_feat = np.concatenate((trainval_feat, final),axis=1)
        print('train val done')

        seriesA = isolate2[trainingval['piece_id'].isin(train_trainvalsets)].apply(lambda x: np.asarray(x.a))
        array = seriesA.to_numpy()
        final = np.vstack(array)
        train_feat = np.concatenate((train_feat, final),axis=1)
        print('train done')
        
    else:
        isolate1 = pd.Series(valtesting[ind]).apply(lambda x: x.a.flatten())
        isolate2 = pd.Series(trainingval[ind]).apply(lambda x: x.a.flatten())
    
        seriesA = isolate1[valtesting['piece_id'].isin(test_testvalsets)].apply(lambda x: x)
        array = seriesA.to_numpy()
        final = np.vstack(array)
        test_feat = np.concatenate((test_feat, final),axis=1)
        print('test done')

        seriesA = isolate1[valtesting['piece_id'].isin(val_testvalsets)].apply(lambda x: x)
        array = seriesA.to_numpy()
        final = np.vstack(array)
        val_feat = np.concatenate((val_feat, final),axis=1)
        print('val done')

        seriesA = isolate2[trainingval['piece_id'].isin(trainval_trainvalsets)].apply(lambda x: x)
        array = seriesA.to_numpy()
        final = np.vstack(array)
        trainval_feat = np.concatenate((trainval_feat, final),axis=1)
        print('train val done')

        seriesA = isolate2[trainingval['piece_id'].isin(train_trainvalsets)].apply(lambda x: x)
        array = seriesA.to_numpy()
        final = np.vstack(array)
        train_feat = np.concatenate((train_feat, final),axis=1)
        print('train done')
    

acf
test done
val done
train val done
train done
spike_waveform_maxamplitude
test done
val done
train val done
train done
spec_spike_waveform
test done
val done
train val done
train done
spec_freq
test done
val done
train val done
train done
spec_acf
test done
val done
train val done
train done
spec_freq_acf
test done
val done
train val done
train done
early_centroid
test done
val done
train val done
train done
soma_centroid
test done
val done
train val done
train done


In [34]:
isolate1 = pd.Series(valtesting['early_centroid']).apply(lambda x: np.asarray(x))
isolate1 = isolate1.apply(lambda x: np.asarray(x.a))
print(isolate1[0])

[17 16]


In [74]:
isolate2 = pd.Series(trainingval['early_centroid'])
print(isolate2.iloc[5000].a)

test = isolate2.to_list()
print(dir(test))
print(test[0:10])

[0, 26]
['__add__', '__class__', '__class_getitem__', '__contains__', '__delattr__', '__delitem__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__iadd__', '__imul__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__rmul__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', 'append', 'clear', 'copy', 'count', 'extend', 'index', 'insert', 'pop', 'remove', 'reverse', 'sort']
[<file_handling.wrapper object at 0x7f9bec2fc2b0>, <file_handling.wrapper object at 0x7f9bec2fc310>, <file_handling.wrapper object at 0x7f9bec2fc370>, <file_handling.wrapper object at 0x7f9bec2fc3d0>, <file_handling.wrapper object at 0x7f9bec2fc430>, <file_handling.wrapper object at 0x7f9bec2fc490>, <file_handling.wrapper object at 0x7f9bec2fc4f0>, <file_handling.wrapper object at 0x7f9bec2fc550>, <file_h

## Check Features are All Good

In [47]:
print(train_feat.shape)
print(trainval_feat.shape)
print(test_feat.shape)
print(val_feat.shape)
print(valtesting.iloc[0])
print(train_feat[0,:])



(115163, 590)
(3786, 592)
(7980, 592)
(8577, 592)
unit_id                                        1.0
dataset_id                     (2012-09-27-3, 003)
run_id                                         003
piece_id                              2012-09-27-3
valid                                         True
label_manual_text_input                OFF parasol
spike_duration                           -0.000372
spike_rate_mean                           2.806329
acf                                           a100
spike_waveform_maxamplitude                   a181
int_piece_id                             -0.050323
int_run_id                                -0.21428
spec_spike_waveform                            a91
spec_freq                                      a91
spec_timeoverlap                                a1
spec_acf                                     a51x1
spec_freq_acf                                  a51
spec_timeoverlap_acf                        0.0025
retinal_eccentricity            

## Save arrays